Pour récupérer dans le code la liste des 2315 mots pouvant être solution :

In [ ]:
import pandas as pd
url1 = 'https://gist.githubusercontent.com/cfreshman/a03ef2cba789d8cf00c08f767e0fad7b/raw/5d752e5f0702da315298a6bb5a771586d6ff445c/wordle-answers-alphabetical.txt'
liste_solutions = pd.read_csv(url1,header=None)
liste_solutions = liste_solutions[0].values.tolist()

Et pour la liste des mots 10657 mots autorisés (ne comprenant pas les mots solutions) :

In [ ]:
url2 = 'https://gist.githubusercontent.com/cfreshman/cdcdf777450c5b5301e439061d29694c/raw/de1df631b45492e0974f7affe266ec36fed736eb/wordle-allowed-guesses.txt'
liste_mots = pd.read_csv(url2,header=None)
liste_mots = liste_mots[0].values.tolist()

In [ ]:
import pandas as pd
import numpy as np
	

def obtenir_motif(mot: str,solution: str) -> str:
    motif_l = [0]*5
    sol = list(solution)
    indices = list(range(5))
    for i in indices[:]:
        if mot[i] == solution[i]:
            motif_l[i]='V'
            sol.remove(mot[i])
            indices.remove(i)
    for i in indices:
        if mot[i] in sol:
            motif_l[i] = 'J'
            sol.remove(mot[i])
        else:
            motif_l[i] = 'G'
    motif = ''
    for s in motif_l:
        motif += s
    return motif

def conv_motif_nb(motif: str) -> int:
    nb = 0
    rg = 0
    for c in motif[::-1]:
        for i in range(3):
            if c=='GJV'[i]:
                nb += i*3**rg
        rg += 1
    return nb

def entropie(mot_test: str,liste_mots: list) -> tuple:
    n = len(liste_mots)
    liste_mots_par_motifs = [[] for _ in range(3**5)]
    for mot in liste_mots:
        liste_mots_par_motifs[conv_motif_nb(obtenir_motif(mot_test,mot))]+=[mot]
    res = 0
    for L in liste_mots_par_motifs:
        X = len(L)
        if X:
            res += -X/n*np.log2(X/n)
    return res,liste_mots_par_motifs

def resultats_depart(liste_depart: list) -> list:
    Resultats = []
    for i in range(len(liste_depart)):
        s,liste_indices_mots_par_motifs = entropie(liste_depart[i],liste_depart)
        Resultats.append((s,i,liste_indices_mots_par_motifs))
    return Resultats

def resultats(liste_depart: list,liste_mots_restants: list) -> list:
    Resultats = []
    for mot in liste_depart:
        s,liste_mots_par_motifs = entropie(mot,liste_mots_restants)
        Resultats.append((s,mot,liste_mots_par_motifs))
    return Resultats

# from random import choice
# mot_mystere = choice(liste)
# print(mot_mystere)

#####################################################
# fabrication de la première liste de résultats (gagne du temps de l'importer par la suite)
# grosse liste
# url = 'https://gist.githubusercontent.com/cfreshman/cdcdf777450c5b5301e439061d29694c/raw/de1df631b45492e0974f7affe266ec36fed736eb/wordle-allowed-guesses.txt'
# Liste des solutions (petite liste)
# url = 'https://gist.githubusercontent.com/cfreshman/a03ef2cba789d8cf00c08f767e0fad7b/raw/5d752e5f0702da315298a6bb5a771586d6ff445c/wordle-answers-alphabetical.txt'
# dataf = pd.read_csv(url,header=None)
# liste_mots_dep = dataf[0].values.tolist()
# Resultats = resultats_depart(liste_mots_dep[:])
# import csv
# with open('liste_base.csv','w') as f:
#     write = csv.writer(f)
#     write.writerows(Resultats)
#####################################################

# importation liste des mots solution 
# (liste obtenue grace au code ci-dessus)
url = 'https://gist.githubusercontent.com/cfreshman/a03ef2cba789d8cf00c08f767e0fad7b/raw/5d752e5f0702da315298a6bb5a771586d6ff445c/wordle-answers-alphabetical.txt'
dataf = pd.read_csv(url,header=None)
liste_mots_dep = dataf[0].values.tolist()
# importation des premiers résultats
url2 = 'http://cordier-phychi.toile-libre.org/divers/download/liste_base.csv'
dataf_result = pd.read_csv(url2,header=None)
liste_resultats = dataf_result.values.tolist() 
import json
Resultats = []
for L1 in liste_resultats:
    L = []
    a,b,c = L1
    k = 0
    c = json.loads(c)
    for L2 in list(c):
        L.append([])
        for i in L2:
            L[k].append(liste_mots_dep[int(i)])
        k += 1
    Resultats.append((float(a),liste_mots_dep[int(b)],L))

#####################################################
# Si démarrage sans la première liste de résultas, décommenter les lignes qui suivent 
# liste des mots solutions
# url = 'https://gist.githubusercontent.com/cfreshman/a03ef2cba789d8cf00c08f767e0fad7b/raw/5d752e5f0702da315298a6bb5a771586d6ff445c/wordle-answers-alphabetical.txt'
# dataf = pd.read_csv(url,header=None)
# liste = dataf[0].values.tolist()
# print("\nPatience... Le premier calcul est un peu long (environ 50 s)\n")
# Resultats = resultats(liste[:],liste[:])
essais = 1
essai1 = max(Resultats)[1].upper()
print("\n")
print(f"Tentez le mot '{essai1}'")
print()
motif_obtenu = input("puis entrez le motif obtenu\nsous la forme d'un mot de 5 lettres\nchoisies parmi 'g', 'j' et 'v' où\n'g' désigne une lettre grise,\n'j' une lettre jaune et\n'v' une lettre verte\n\nmotif : ").upper()
while True or essais <= 5:
  liste_mots = max(Resultats)[2][conv_motif_nb(motif_obtenu)]
  if len(liste_mots) == 1:
    print("\n"+" "*4+"-"*9)  
    print('-->',end = ' ')
    print(f"| {liste_mots[0].upper()} |")
    print(" "*4+"-"*9+"\n")  
    break
  Resultats = resultats(liste_mots_dep,max(Resultats)[2][conv_motif_nb(motif_obtenu)])
  nvessai = max(Resultats)[1].upper()
  print()
  print(f"Tentez le mot '{nvessai}'")
  print()
  motif_obtenu = input("motif : ").upper()
  essais += 1
  
#####################################################
# pour tester sur tous les mots
# url = 'https://gist.githubusercontent.com/cfreshman/a03ef2cba789d8cf00c08f767e0fad7b/raw/5d752e5f0702da315298a6bb5a771586d6ff445c/wordle-answers-alphabetical.txt'
# liste = pd.read_csv(url,header=None)
# liste = liste[0].values.tolist()
# succes = 0
# Distrib = [0]*6
# nb_moy_tent = 0
# count = 0
# print("Patience... Le premier calcul est un peu long (environ 30 s)\n")
# Res = resultats(liste[:],liste[:])[:]
# for mot_mystere in liste:
#     essais = 1
#     conv_motif_nb
#     Resultats = Res[:]
#     motif_obtenu = obtenir_motif('raise',mot_mystere)
#     if mot_mystere == 'raise':
#         Distrib[0] += 1
#         nb_moy_tent += essais
#         print(count,mot_mystere,essais)
#         count += 1
#         continue
#     while essais <= 6:
#         liste_mots = max(Resultats)[2][conv_motif_nb(motif_obtenu)]
#         Resultats = resultats(liste,liste_mots)  
#         motif_obtenu = obtenir_motif(max(Resultats)[1],mot_mystere)
#         essais += 1
#         if motif_obtenu == "VVVVV":
#             succes += 1
#             Distrib[essais-1] += 1
#             nb_moy_tent += essais
#             print(count,mot_mystere,essais)
#             break
#         if len(liste_mots) == 1:
#             succes += 1
#             Distrib[essais-1] += 1
#             nb_moy_tent += essais
#             print(count,mot_mystere,essais)
#             break
#     count += 1
#####################################################
